# JSON

- moduł json (wbudowany)

In [ ]:
from __future__ import print_function
import json


d = {'a': 1, 'b': 2}

d_json = json.dumps(d)  # zrzuca JSONa do stringu
print(d_json)

d_json = json.dumps(d, indent=4)  # dodaje wcięcia do JSONA
print(d_json)

d2 = json.loads(d_json)  # wczytuje JSONa ze stringu
print(d2 == d)

# json.dump(open('plik.json', 'w'), d) - zrzuca json do pliku - plik można zastąpić przez StringIO lub BytesIO
# d3 = json.load(open('plik.json'))  # wczytuje json z pliku

In [ ]:
# można dodać dekoder dla słowników
import json

def decoder(obj):
    print(obj)
    ...
    return(obj)

d = {'a': 1, 'b': 2}
dj = json.dumps(d)

d2 = json.loads(dj, object_hook=decoder)
print(d2)

# dekoder wcale nie musi zwracać słownika
def decoder(obj):
    print(obj)
    ...
    return lambda: None

d = {'a': 1, 'b': 2}
dj = json.dumps(d)

d2 = json.loads(dj, object_hook=decoder)
print(d2)

In [ ]:
# zrzucanie
import json


class Encoder(json.JSONEncoder):
    def default(self, o):      # default jest wykonywane kiedy mamy typ, którego json.dumps
        print('encoding: ', o)  # sam nie zakoduje
        if isinstance(o, set):
            o = list(o)
        return json.dumps(o)
    

def decoder(obj):
    print('decoding: ', o)
    return set(obj)

def encoder(o):
    print('encoding: ', o)  # sam nie zakoduje
    if isinstance(o, set):
        o = list(o)
    return json.dumps(o)


d = set([1, 2, 3])
# dj = Encoder().encode(d)           # można zakodowywać bezpośrednio klasą kodującą
# dj = json.dumps(d, cls=Encoder)    # można przekazać klasę kodującą jako parametr cls
dj = json.dumps(d, default=encoder)  # można przekazać funkcję jako parametr default
# default się wywołuje kiedy json sam nie jest w stanie zrzucić obiektu - powinno zwracać jakiś
# obiekt lub wyrzucać TypeError

d2 = json.loads(dj, object_hook=decoder)
print(d2)
    

do ładnego wypisywania JSONa z poziomu terminala można użyć json.tool:

cat plik.json | python -m json.tool

# XML

 - moduł xml (wbudowany)

## moduł xml

In [ ]:
from xml.dom import minidom

content = """
<package>
	<module>
		<version architecture="AX">15.5</version>
		<component>APG</component>
	</module>
	<module>
		<version architecture="CX">13.3</version>
		<component>APZ</component>
	</module>
</package>
"""

# tree = minidom.parse(path)  # otiwera xml z pliku
root = minidom.parseString(content)

child_nodes = root.childNodes
print(child_nodes)

modules = root.getElementsByTagName("module")
print(modules)
for module in modules:
    print("child nodes of module ", module, ': ', module.childNodes)
    for node in module.childNodes:
        print("node name: ", node.nodeName)
        if node.nodeName == 'version':
            arch = node.getAttribute('architecture')
            print("architecture:", arch)
        elif node.nodeName == 'component':
            spam = node.getAttribute("spam")  # get nonexisting attribute
            print('spam:', spam)
        content = node.toxml()
        print(content)

In [ ]:
from xml.etree import ElementTree as ET

content = """
<package>
	<module>
		<version architecture="AX">15.5</version>
		<component>APG</component>
	</module>
	<module>
		<version architecture="CX">13.3</version>
		<component>APZ</component>
	</module>
</package>
"""

root = ET.fromstring(content)
print(root)
for module in root:
    version, component = list(module)
    print(version, version.attrib, version.text)
    print(component, component.attrib, component.text)

    
print(50 * '=')
for version in root.iter('version'):  # iteruje po wszystkich tagach version
    print(version, version.text)
    
    
print(50 * '=')
for version in root.findall('version'):  # iteruje po tagach version będących
    print(version, version.text)         # bezpośrednimi dziecmi
    
    
for component in root.iter('component'):
    component.set('legacy', 'true')  # do każdego komponentu ustaw atrybur legacy na true


print(ET.dump(root))  # zrzuć tag na xml - widać zmienione komponenty

print(50 * '=')
for module in root:
    if module[1].text == "APZ": # tag component
        root.remove(module)
        
print(ET.dump(root))  # zrzuć tag na xml - widać usunięty moduł

print(50 * '=')
new_module = ET.Element('module')
new_version = ET.SubElement(new_module, 'version')
new_version.attrib['architecture'] = 'AY'
new_version.text = '15.8'
new_component = ET.SubElement(new_module, 'component')
new_component.text = 'CP'

root.append(new_module)
print(ET.dump(root))

# YAML

- pyyaml (moduł zewnętrzny)

## podstawowe operacje

In [ ]:
import yaml


document = """
a: 1
b:
  c: 3
  d: 4
e:
  - f
  - g
"""

doc = yaml.load(document)
print(doc)

document = yaml.dump(doc)  # nie przetwarza zagnieżdżonych
print(document)

# default_flow_style=False zawsze serializuje do postaci bolokowej
document = yaml.dump(doc, default_flow_style=False)
print(document)


## Ładowanie YAMLów

In [ ]:
import yaml

# --- to separator dokumentów
documents = """
---
a: b
c: d
---
e: f
g: h
---
i: j
k: l
"""

print(list(yaml.load_all(documents)))

In [ ]:
# yaml pozwala tworzyć obiekty pythonowe na podstawie zawartości
from pprint import pprint
import yaml

document = """
#komentarz
none: null
bool: [true, false, on, off]
int: 22
float: 33.3
lista: [a, b, c]
slownik: {a: 1, b: 2}
"""

pprint(yaml.load(document))

In [ ]:
# pyaml pozwala importować pythonowe obiekty
import yaml


class A(object):
    def __init__(self, a, b, c):
        print('A.__init__')
        self._x = a
        self._y = b
        self._z = c
        

document = """
!!python/object:__main__.A
x: 1
y: 2
z: 3
"""

# yaml nie inicjalizuje klasy podanymi parametrami
# tylko przypisuje je jezpośrednio do instancji
a = yaml.load(document)
print(dir(a))

try:
    # safe load nei pozwala na ładowanie innych obieków niz wbudiwane - kwestia bezpieczeństwa
    a = yaml.safe_load(document)
except yaml.constructor.ConstructorError:
    print("Nie mozna stworzyc obiektu")

## Zrzucanie YAMLów

funkcja dump przyjmuje opcjonalnie drugi argument, który jest otwartym plikiem do którego bedzie zrzucany yaml (jażeli nie podany to yaml zostanie zwrócony z funkcji)

In [ ]:
import yaml

obiekty = [{'a': 1}, [3], 23, '44']


# dump_all przyuje sekwencję/generator zwracający obiekty do serializacji
# drugi (opcjonalny) argument to otwarty plik
print(yaml.dump_all(obiekty))

In [ ]:
import yaml


# żeby móc zdefiniować własny tag, trzeba dziedziczyć po YAMLObject
class A(yaml.YAMLObject):
    yaml_tag = '!A'  # własny tag
    
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
        
print(yaml.dump(A(1, 2, 3), default_flow_style=False))